In [6]:
setwd("/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/Summary_files/")
file_name <- "RSG_SAGs_O2_consumption_2018-2019.csv"
all_data_a <- read.csv(file=file_name, header=TRUE)
#column names:
  #seq_order	
  #name	
  #gen	
  #Predicted_respiration_rate_fmol_O2perhr	
  #Sample_collection_Date


In [5]:
unique_dates <- unique(all_data_a$Sample_collection_Date)
for (i_date in 1:(1+length(unique_dates))){  #
  if (i_date == 1){
    
    #get list of gens to keep and plot
    file_name <- "Genera_with_10+_SAGs.txt"
    sag_10 <- read.csv(file=file_name, header=FALSE)
    #column names:
    #gen_keep
    
    #get avg_weighted_O2_consumption for each gen then sort by that
    file_name <- "avg_cells_O2_consumption_per_GEN_2018-2019.csv"
    avg_O2 <- read.csv(file=file_name, header=TRUE)
    #avg_O2[avg_O2 == "Unclassified"] <- 999
    #column names:
    #unknown	
    #gen	  
    #avg_cells_per_ml_2018_2019
    #avg_weighted_O2_consumption
    re_order <- order(-avg_O2$avg_cells_per_ml_2018_2019)
    gen_order <- data.frame(avg_O2$gen[re_order])
    colnames(gen_order) <- "genus"
    cells_per_ml_order <- avg_O2$avg_cells_per_ml_2018_2019[re_order]
    avg_weighted_O2 <- avg_O2$avg_weighted_O2_consumption[re_order]
    df <- data.frame(gen_order, cells_per_ml_order, avg_weighted_O2)
    colnames(df) <- c("genus", "cells_per_ml", "avg_O2_consumed")
    
    #check that gen_order is in gen_keep
    gen_order_2 <- c()
    for (i in seq_along(sag_10$gen_keep)){
      keep <- which(sag_10$gen_keep[i] == gen_order)
      if(keep > 0){gen_order_2[keep] <- sag_10$gen_keep[i]}
    }
    keep <- !is.na(gen_order_2)
    gen_order <- gen_order_2[keep]
    
    all_data <- all_data_a
    main_title <- "All Data"
    
  } else {
    #get daily average cells/ml and average O2 consumption
    keep <- all_data_a$Sample_collection_Date == unique_dates[i_date-1]
    all_data <- all_data_a[keep,]
    
    file_name <- "avg_O2_consumption_and_abundance_per_day_melt.csv"
    daily_data <- read.csv(file=file_name, header=TRUE)
    keep <- (daily_data$date-20000000) == unique_dates[i_date-1]
    days_data <- daily_data[keep,]
    df <- data.frame(days_data$gen, 
                     days_data$cells_per_ml,
                     days_data$avg_fmol_O2_consumed)
    colnames(df) <- c("genus","cells_per_ml","avg_O2_consumed")
    go_dataframe <- data.frame(gen_order)
    colnames(go_dataframe) <- "gen_order"
    df <- merge(go_dataframe, df, sort=FALSE, by.x="gen_order", by.y="genus",
                all.y=FALSE, all.x=TRUE)
    colnames(df) <- c("genus","cells_per_ml","avg_O2_consumed")
    
    main_title <- unique_dates[i_date-1]
  }
  
  gen_data <- all_data$gen
  O2_data <- all_data[,4]
  df2 <- merge(df, sag_10, by.x="genus", by.y="gen_keep", 
               all.y=TRUE, all.x=FALSE, sort=FALSE)

#create matrix for box plot
#need columns in correct order and only keep desired gens
plot_data <- matrix(NA, ncol=length(sag_10$gen_keep), nrow=900)
colnames(plot_data) <- gen_order

for (i in seq_along(sag_10$gen_keep)){
  #keep the data for this gen
  keep <- (gen_data == sag_10$gen_keep[i])  
  #if this gen exists, store it in the right column
  i_col <- which(sag_10$gen_keep[i] == gen_order) 
  if (sum(keep, na.rm=TRUE) > 0){ 
    plot_data[1:sum(keep, na.rm=TRUE),i_col] <- log10(O2_data[keep])
  }
}
i_col <- length(sag_10$gen_keep)

#set file for graphic output
plot_file <- paste("Box plot ", main_title," 2.pdf", sep="")
pdf(file=plot_file, height=4.5, width=6.0)

#store original and then set new plotting margins
op <- par(no.readonly=TRUE)
par(mar = c(5.3, 4.3, 4.3, 4))

#box plot
y_label_text <- expression("log"[10]*"(O"[2]*" consumption)")
abc <- boxplot(plot_data, names=NA, na.action=NA,
        xlab="", ylab=y_label_text,
        cex.lab=0.9, main="",
        xaxs="i", yaxs="i", xlim=c(0,(i_col+1)), ylim=c(-3,2))
ax_lim <- par("usr")
y_text <- ax_lim[3]-0.05*(ax_lim[4]-ax_lim[3])
text(x=1:i_col, y=rep(y_text, times=i_col), 
     labels=colnames(plot_data)[1:i_col], cex=0.5, 
     xpd=NA, srt=90, adj=c(1,0.5))
mtext("Genus", side=1, line=1.4, cex=0.9)
mtext(main_title, side=3, line=0.5, cex=1., font=2)

#first dot plot - avg weighted O2 consumption
plot_pch <- rep(16,times=length(df2[,3]))
df2_min <- round(min(df2[,3], na.rm=TRUE),10)
change <- (round(df2[,3],10) <= df2_min)
plot_pch[change] <- 1
points(x=1:i_col, y=log10(df2[,3]), pch = plot_pch, col = "red", cex=0.6)

#second point plot - avg cells per ml using 2nd y-axis
par(new = TRUE)
plot(x=1:i_col, y=df2[,2], xlim=c(0,(i_col+1)),ylim=c(0,4e5), cex=0.5,
     pch = 17, col = "green", axes = FALSE, xlab = "", ylab = "")
axis(side = 4, at = (0:4)*1e5)
mtext(expression("Avg cell ml"^-1), 
      side = 4, line = 2.5, cex=0.9)

#legend for point plots
legend(x="topright",legend=rbind("Cells per ml",
       "Average weighted O2 consumption"),
       col=c("green","red"), pch=c(17,18), cex=0.6)

dev.off()
#reset graphic parameters
par(op)
}

ERROR: Error in -avg_O2$avg_cells_per_ml_2018_2019: invalid argument to unary operator
